In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls
%cd  drive/MyDrive/adversarial-attacks-pytorch/

Mounted at /content/drive
drive  sample_data
/content/drive/MyDrive/adversarial-attacks-pytorch


In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM
from demos.models import CNN

In [3]:
print("PyTorch", torch.__version__)
print("Torchvision", torchvision.__version__)
print("Torchattacks", torchattacks.__version__)
print("Numpy", np.__version__)

PyTorch 1.11.0+cu113
Torchvision 0.12.0+cu113
Torchattacks 3.2.6
Numpy 1.21.6


In [4]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [5]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,)),
    AddGaussianNoise(0., 1.)]),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]),
                         download=True)

In [6]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False,)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

In [7]:

model = CNN().cuda()
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
atk = PGD(model, eps=0.3, alpha=0.1, steps=7)

In [16]:
num_epochs  =5
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    ct = 0
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        #X = atk(batch_images, batch_labels).cuda()
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        #print(pre.shape,Y.shape)
        cost = loss(pre, Y)
        ct += (torch.max(pre.data, 1)[-1] == Y).sum()
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
    print(f"Accuracy at epoch {epoch} is {ct/len(mnist_train)}")


Epoch [1/5], lter [100/468], Loss: 0.0299
Epoch [1/5], lter [200/468], Loss: 0.0369
Epoch [1/5], lter [300/468], Loss: 0.0322
Epoch [1/5], lter [400/468], Loss: 0.0039
Accuracy at epoch 0 is 0.9891499876976013
Epoch [2/5], lter [100/468], Loss: 0.0214
Epoch [2/5], lter [200/468], Loss: 0.0432
Epoch [2/5], lter [300/468], Loss: 0.0829
Epoch [2/5], lter [400/468], Loss: 0.0132
Accuracy at epoch 1 is 0.9902333617210388
Epoch [3/5], lter [100/468], Loss: 0.0439
Epoch [3/5], lter [200/468], Loss: 0.0622
Epoch [3/5], lter [300/468], Loss: 0.0209
Epoch [3/5], lter [400/468], Loss: 0.0031
Accuracy at epoch 2 is 0.9914000034332275
Epoch [4/5], lter [100/468], Loss: 0.0224
Epoch [4/5], lter [200/468], Loss: 0.0473
Epoch [4/5], lter [300/468], Loss: 0.1338
Epoch [4/5], lter [400/468], Loss: 0.0018
Accuracy at epoch 3 is 0.9905000329017639
Epoch [5/5], lter [100/468], Loss: 0.0069
Epoch [5/5], lter [200/468], Loss: 0.0262
Epoch [5/5], lter [300/468], Loss: 0.0179
Epoch [5/5], lter [400/468], Loss:

In [17]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 99.11 %


In [18]:
from torchattacks import *
model.eval()

correct = 0
total = 0

atk = FGSM(model, eps=0.3)
atk1 = torchattacks.FGSM(model, eps=3)
atk2 = torchattacks.PGD(model, eps=8/255, alpha=2/200, steps=40, random_start=True)
atk = torchattacks.MultiAttack([atk1, atk2])

atks = [
    torchattacks.FGSM(model, eps=8/255),
    torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=100),
    torchattacks.RFGSM(model, eps=8/255, alpha=2/255, steps=100),
    torchattacks.CW(model, c=1, lr=0.01, steps=100, kappa=0),
    torchattacks.PGD(model, eps=8/255, alpha=2/225, steps=100, random_start=True),
    torchattacks.PGDL2(model, eps=1, alpha=0.2, steps=100),
    torchattacks.EOTPGD(model, eps=8/255, alpha=2/255, steps=100, eot_iter=2),
    torchattacks.FFGSM(model, eps=8/255, alpha=10/255),
    TPGD(model, eps=8/255, alpha=2/255, steps=100),
    MIFGSM(model, eps=8/255, alpha=2/255, steps=100, decay=0.1),
    VANILA(model),
    GN(model, std=0.1),
    APGD(model, eps=8/255, steps=100, eot_iter=1, n_restarts=1, loss='ce'),
    APGD(model, eps=8/255, steps=100, eot_iter=1, n_restarts=1, loss='dlr'),
    APGDT(model, eps=8/255, steps=100, eot_iter=1, n_restarts=1),
    FAB(model, eps=8/255, steps=100, n_classes=10, n_restarts=1, targeted=False),
    FAB(model, eps=8/255, steps=100, n_classes=10, n_restarts=1, targeted=True),
    Square(model, eps=8/255, n_queries=5000, n_restarts=1, loss='ce'),
    AutoAttack(model, eps=8/255, n_classes=10, version='standard'),
    OnePixel(model, pixels=5, inf_batch=50),
    DeepFool(model, steps=100),
    DIFGSM(model, eps=8/255, alpha=2/255, steps=100, diversity_prob=0.5, resize_rate=0.9)
]


#atk = torchattacks.MultiAttack(atks)

for images, labels in test_loader:
    
    images = atk(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 21.26 %
